## Demonstração Graph Neural Networks

In [1]:
# ganrante que as bibliotecas necessárias estrão disponíveis no jupyter
# %conda install -c pytorch pytorch torchvision
# %conda install pyg -c pyg -c conda-forge

### Recursos básicos da Pytorch Geometric

In [2]:
#Somente para imprimir imagens locais no jupyter-notebook
from IPython import display

import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
data

Data(x=[3, 1], edge_index=[2, 4])

In [3]:
# display.Image("imgs/grafosimples.png")

### O objeto Data

In [4]:
print(data.keys)

print()
print(data['x'])

print()
for key, item in data:
    print(key+" presente no grafo")

<bound method BaseData.keys of Data(x=[3, 1], edge_index=[2, 4])>

tensor([[-1.],
        [ 0.],
        [ 1.]])

x presente no grafo
edge_index presente no grafo


In [5]:
print(data.num_nodes)

3


In [6]:
print(data.num_edges)


print(data.num_node_features)


print(data.has_isolated_nodes())


4
1
False


In [7]:
# Tranferir o objeto data para a GPU.
device = "cuda" if torch.cuda.is_available() else "cpu"

data = data.to(device)

### Datasets prontos disponíveis na Pytorch Geometric
Existem vários datasets já disponíveis. É útil olhar para esses "formatados", pois podemos entender como os nossos datasets podem estar depois de carregados apropriadamente.

In [8]:
from torch_geometric.datasets import TUDataset

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES')
dataset

ENZYMES(600)

In [9]:
len(dataset)


600

In [10]:
dataset[500]


Data(edge_index=[2, 160], x=[67, 3], y=[1])

In [11]:
dataset.num_classes


6

In [12]:
dataset.num_node_features

3

In [13]:
#Se necessário, podemos embaralhar o dataset
dataset = dataset.shuffle()

In [14]:
#Podemos fatiar o dataset
train_dataset = dataset[:500]
train_dataset

ENZYMES(500)

In [15]:
test_dataset = dataset[500:]
test_dataset

ENZYMES(100)

### Exemplo de dataset com um único grafo
Este exemplo ilustra um grafo para os problemas de classificação de nós

In [16]:
from torch_geometric.datasets import Planetoid

dataset = Planetoid(root='/tmp/Cora', name='Cora')
dataset

Cora()

In [17]:
print(len(dataset))

print(dataset.num_classes)

print(dataset.num_node_features)

1
7
1433


In [18]:
data = dataset[0]
data

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])

Veja que tem os campos train_mask, val_mask e test_mask. Isso facilita a realização do treinamento e testes. <br>

In [19]:
print(data.train_mask.sum().item())

print(data.val_mask.sum().item())

print(data.test_mask.sum().item())

140
500
1000


train_mask: indica quais nós serão usados para treinar (140 nós)<br>

val_mask: nós para validação (500 nós),

test_mask: quais nós para testar (1000 nós).

### Mini-batches

Abordagem comum em redes neurais.

PyG contém seu próprio torch_geometric.loader.DataLoader, que já cuida desse processo.

In [20]:
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

dataset = TUDataset(root='/tmp/ENZYMES', name='ENZYMES', use_node_attr=True)
loader = DataLoader(dataset, batch_size=100, shuffle=True)

for batch in loader:
    print(batch)
    print("Numero de grafos:"+str(batch.num_graphs))
    
    #O atributo batch é um vetor que indica em que grafo cada nó pertence
    print(batch.batch)
    print()
    

DataBatch(edge_index=[2, 11720], x=[3042, 21], y=[100], batch=[3042], ptr=[101])
Numero de grafos:100
tensor([ 0,  0,  0,  ..., 99, 99, 99])

DataBatch(edge_index=[2, 12132], x=[3277, 21], y=[100], batch=[3277], ptr=[101])
Numero de grafos:100
tensor([ 0,  0,  0,  ..., 99, 99, 99])

DataBatch(edge_index=[2, 12764], x=[3312, 21], y=[100], batch=[3312], ptr=[101])
Numero de grafos:100
tensor([ 0,  0,  0,  ..., 99, 99, 99])

DataBatch(edge_index=[2, 11990], x=[3180, 21], y=[100], batch=[3180], ptr=[101])
Numero de grafos:100
tensor([ 0,  0,  0,  ..., 99, 99, 99])

DataBatch(edge_index=[2, 12406], x=[3257, 21], y=[100], batch=[3257], ptr=[101])
Numero de grafos:100
tensor([ 0,  0,  0,  ..., 99, 99, 99])

DataBatch(edge_index=[2, 13552], x=[3512, 21], y=[100], batch=[3512], ptr=[101])
Numero de grafos:100
tensor([ 0,  0,  0,  ..., 99, 99, 99])



O atributo batch nesse objeto acima é um vetor que indica em que grafo cada nó pertence

### Mais sobre datasets

##### Não é necessário usar a interface dataset, por exemplo, quando quiser criar dados sintéticos dinamicamente, sem salvá-los explicitamente no disco. Neste caso, simplesmente passe uma lista python regular contendo objetos Data e passe-os para o DataLoader:

In [21]:
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

data_list = [Data(...), ..., Data(...)]
loader = DataLoader(data_list, batch_size=10)

A biblioteca DeepSnap pode user útil para criar e trabalhar com datasets no Pytorch Geometric

In [22]:
import networkx as nx
from deepsnap.graph import Graph
from deepsnap.dataset import GraphDataset

G = nx.complete_graph(100)
H1 = Graph(G)
H2 = H1.clone()
H3 = H1.clone()
H4 = H1.clone()
H5 = H1.clone()
H6 = H1.clone()
dataset = GraphDataset(graphs=[H1, H2, H3, H4, H5, H6], task = 'graph')
len(dataset)


/mnt/wsl/PHYSICALDRIVE2/Utfpr/TCC/TCC_graphs_study/.venv/lib/python3.11/site-packages/deepsnap/graph.py:522: UserWarning: Node related key is required.
  warnings.warn("Node related key is required.")


6

Separando em treino e teste com o deepSNAP

In [23]:
train, val, test = dataset.split(transductive=False, split_ratio=[0.8, 0.1, 0.1])
print(train)
print(val)

GraphDataset(3)
GraphDataset(1)


#### Indutive vs transdutive 
'indutive' (para datasets com vários gráficos) divide o dataset por grafos.Conjuntos distintos de grafos são usados para treinamento, validação e teste, e os gráficos de teste nunca  são vistos durante o treinamento. Isso pode ser feito para tarefas em nível de nó, borda e gráfico. 

'transdutive', todos os gráficos são vistos durante o tempo de treinamento, mas os rótulos de determinados nós e arestas não são observados no treinamento e são usados para validação e teste. Isso se aplica a tarefas em nível de nó e aresta.